# 神经网络模型：

In [124]:
import torch

class Para:
    # tensor_board_log_dir = 'runs/exp0'
    feature_column_start_name = 'ep_ratio_ttm'
    feature_column_end_name = 'BR'

    # 模型设置
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    classification = 2 # 2, 3

    # 权重
    cross_weight = list()
    if classification == 3:
        cross_weight = [1.0, 1.0 ,1.0]
    elif classification == 2:
        cross_weight = [1.0, 1.0]
    elif classification == 5:
        cross_weight = [1.0, 1.0, 1.0, 1.0, 1.0]

    batch_size = 16
    lr = 1e-3
    drop = 0.5
    epochs = 60

    # 数据集设置
    month_in_sample = range(0, 1)
    # month_test = range(36, 48)

    percent_cv = 0.1 # 10% cross validation

    data_path = 'data/mt_space_1d_rate_20d_12-21_pre'


    seed = 2022
    torch.manual_seed(seed)

    info_str0 = data_path[5:7]+'_'+'c'+str(classification)+'_s'+str(percent_cv)
    info_str1 = '_b'+str(batch_size)+'_lr'+str(lr)+'_d'+str(drop)+'_e'+str(epochs)
    info_str = info_str0 + info_str1

    save_model_path = 'models/'+'model_'+info_str+'.pth'

para = Para()
print(para.info_str)
print(para.save_model_path)
print(torch.cuda.is_available())
print(torch.cuda.device_count())
if torch.cuda.is_available():
    print(torch.cuda.get_device_name(0))
    print(torch.cuda.current_device())

mt_c2_s0.1_b16_lr0.001_d0.5_e60
models/model_mt_c2_s0.1_b16_lr0.001_d0.5_e60.pth
True
1
GeForce MX150
0


### 构建训练集

In [125]:
import numpy as np
import pandas as pd

data_in_sample = None
for i_month in para.month_in_sample:
    file_name = para.data_path + '/' + str(i_month) + '.csv'
    data_curr_month = pd.read_csv(file_name)

    data_curr_month = data_curr_month.dropna(axis=0)

    data_curr_month.insert(loc=0, column='return_bin', value=np.nan)

    data_curr_month.loc[data_curr_month['yield_rate']>0, 'return_bin'] = 0
    data_curr_month.loc[data_curr_month['yield_rate']<=0, 'return_bin'] = 1

    if i_month == para.month_in_sample[0]:
        data_in_sample = data_curr_month
    else:
        data_in_sample = pd.concat([data_in_sample, data_curr_month])
        # data_in_sample = data_in_sample.append(data_curr_month)

data_in_sample

,return_bin,order_book_id,board_type,sector_code,month,date,yield_rate,ep_ratio_ttm,pb_ratio_ttm,sp_ratio_ttm,...,RSI10,SY,BIAS20,VOL30,VOL60,VOL120,VOLT20,VOLT60,AR,BR
0,1.0,600519.XSHG,MainBoard,ConsumerStaples,0,2012-01-04,-0.121926,-0.327793,0.119714,-0.281492,...,-1.516255,0.287582,-1.499037,0.243383,-0.108178,-0.064657,-0.714641,-0.765841,-1.286900,-1.005713
1,0.0,600519.XSHG,MainBoard,ConsumerStaples,1,2012-01-05,0.164856,-0.308310,0.092927,-0.260658,...,-1.570016,0.287582,-1.622616,0.227465,-0.093272,-0.060556,-0.695599,-0.763324,-1.420785,-1.197929
2,1.0,600519.XSHG,MainBoard,ConsumerStaples,2,2012-01-06,-0.071459,-0.340148,0.137024,-0.294705,...,-1.041819,0.287582,-1.152234,0.209195,-0.100443,-0.063556,-0.697842,-0.760783,-1.311010,-1.105942
3,0.0,600519.XSHG,MainBoard,ConsumerStaples,3,2012-01-09,0.020061,-0.352324,0.154334,-0.307724,...,-1.155946,0.287582,-0.927281,0.208047,-0.097007,-0.059870,-0.700386,-0.759030,-1.307988,-1.151213
4,1.0,600519.XSHG,MainBoard,ConsumerStaples,4,2012-01-10,-0.346104,-0.407505,0.236085,-0.366732,...,-0.648100,1.031276,-0.200881,0.205168,-0.055170,-0.046756,-0.704483,-0.759990,-1.042536,-1.143190
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2400,0.0,600519.XSHG,MainBoard,ConsumerStaples,2406,2021-11-29,0.534437,-1.136781,1.505894,-1.190443,...,1.818819,1.031276,1.512096,-0.824439,-0.534328,-0.416452,2.195698,1.699644,0.822318,0.443818
2401,0.0,600519.XSHG,MainBoard,ConsumerStaples,2407,2021-11-30,0.915285,-1.112226,1.404070,-1.165418,...,0.873176,1.031276,0.815918,-0.830065,-0.533581,-0.409498,2.318976,1.678364,0.533745,0.381303
2402,0.0,600519.XSHG,MainBoard,ConsumerStaples,2408,2021-12-01,0.328554,-1.113259,1.408238,-1.166470,...,1.054461,1.774970,0.741040,-0.834063,-0.556413,-0.412365,2.346959,1.643454,0.429537,0.580646
2403,0.0,600519.XSHG,MainBoard,ConsumerStaples,2409,2021-12-02,0.522385,-1.113728,1.410134,-1.166948,...,1.100556,1.774970,0.675672,-0.838373,-0.577372,-0.415137,2.417174,1.576462,0.646957,0.947160


In [126]:
X_in_sample = data_in_sample.loc[:, para.feature_column_start_name: para.feature_column_end_name]
y_in_sample = data_in_sample.loc[:, 'return_bin']

from sklearn.model_selection import train_test_split

X_train, X_cv, y_train, y_cv = train_test_split(X_in_sample, y_in_sample, test_size=para.percent_cv, shuffle=False) # True, random_state=para.seed)
# X_train, X_cv, y_train, y_cv = train_test_split(X_in_sample, y_in_sample, test_size=para.percent_cv, shuffle=False)

X_train

,ep_ratio_ttm,pb_ratio_ttm,sp_ratio_ttm,MACD_DIFF,MACD_DEA,MACD_HIST,RSI10,SY,BIAS20,VOL30,VOL60,VOL120,VOLT20,VOLT60,AR,BR
0,-0.327793,0.119714,-0.281492,-0.491825,-0.453595,-0.236368,-1.516255,0.287582,-1.499037,0.243383,-0.108178,-0.064657,-0.714641,-0.765841,-1.286900,-1.005713
1,-0.308310,0.092927,-0.260658,-0.524533,-0.473787,-0.284174,-1.570016,0.287582,-1.622616,0.227465,-0.093272,-0.060556,-0.695599,-0.763324,-1.420785,-1.197929
2,-0.340148,0.137024,-0.294705,-0.531742,-0.491461,-0.250539,-1.041819,0.287582,-1.152234,0.209195,-0.100443,-0.063556,-0.697842,-0.760783,-1.311010,-1.105942
3,-0.352324,0.154334,-0.307724,-0.528374,-0.504890,-0.193818,-1.155946,0.287582,-0.927281,0.208047,-0.097007,-0.059870,-0.700386,-0.759030,-1.307988,-1.151213
4,-0.407505,0.236085,-0.366732,-0.493942,-0.508368,-0.060878,-0.648100,1.031276,-0.200881,0.205168,-0.055170,-0.046756,-0.704483,-0.759990,-1.042536,-1.143190
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2159,-1.122708,1.622430,-1.169017,0.203439,0.251474,-0.141590,0.015421,0.287582,-0.120539,-0.886344,-1.019756,-0.943319,0.095696,0.109752,0.042382,0.741488
2160,-1.123224,1.624665,-1.169547,0.199552,0.243039,-0.127109,0.311329,0.287582,-0.131706,-0.878593,-1.027028,-0.940690,0.054954,0.072981,0.155255,1.001939
2161,-1.128802,1.648983,-1.175277,0.240226,0.244872,0.010085,0.153422,1.031276,-0.015087,-0.866933,-1.029284,-0.940762,0.067985,0.044237,-0.108777,0.737335
2162,-1.150504,1.746590,-1.197569,0.463518,0.293449,0.634608,0.552886,1.031276,0.461300,-0.795231,-1.000395,-0.919978,0.185092,0.044332,-0.080123,0.957555


In [127]:
y_train

0       1.0
1       0.0
2       1.0
3       0.0
4       1.0
       ... 
2159    0.0
2160    0.0
2161    0.0
2162    0.0
2163    0.0
Name: return_bin, Length: 2164, dtype: float64

In [128]:
data_train = pd.concat([X_train, y_train], axis=1)
data_cv = pd.concat([X_cv, y_cv], axis=1)
data_train

,ep_ratio_ttm,pb_ratio_ttm,sp_ratio_ttm,MACD_DIFF,MACD_DEA,MACD_HIST,RSI10,SY,BIAS20,VOL30,VOL60,VOL120,VOLT20,VOLT60,AR,BR,return_bin
0,-0.327793,0.119714,-0.281492,-0.491825,-0.453595,-0.236368,-1.516255,0.287582,-1.499037,0.243383,-0.108178,-0.064657,-0.714641,-0.765841,-1.286900,-1.005713,1.0
1,-0.308310,0.092927,-0.260658,-0.524533,-0.473787,-0.284174,-1.570016,0.287582,-1.622616,0.227465,-0.093272,-0.060556,-0.695599,-0.763324,-1.420785,-1.197929,0.0
2,-0.340148,0.137024,-0.294705,-0.531742,-0.491461,-0.250539,-1.041819,0.287582,-1.152234,0.209195,-0.100443,-0.063556,-0.697842,-0.760783,-1.311010,-1.105942,1.0
3,-0.352324,0.154334,-0.307724,-0.528374,-0.504890,-0.193818,-1.155946,0.287582,-0.927281,0.208047,-0.097007,-0.059870,-0.700386,-0.759030,-1.307988,-1.151213,0.0
4,-0.407505,0.236085,-0.366732,-0.493942,-0.508368,-0.060878,-0.648100,1.031276,-0.200881,0.205168,-0.055170,-0.046756,-0.704483,-0.759990,-1.042536,-1.143190,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2159,-1.122708,1.622430,-1.169017,0.203439,0.251474,-0.141590,0.015421,0.287582,-0.120539,-0.886344,-1.019756,-0.943319,0.095696,0.109752,0.042382,0.741488,0.0
2160,-1.123224,1.624665,-1.169547,0.199552,0.243039,-0.127109,0.311329,0.287582,-0.131706,-0.878593,-1.027028,-0.940690,0.054954,0.072981,0.155255,1.001939,0.0
2161,-1.128802,1.648983,-1.175277,0.240226,0.244872,0.010085,0.153422,1.031276,-0.015087,-0.866933,-1.029284,-0.940762,0.067985,0.044237,-0.108777,0.737335,0.0
2162,-1.150504,1.746590,-1.197569,0.463518,0.293449,0.634608,0.552886,1.031276,0.461300,-0.795231,-1.000395,-0.919978,0.185092,0.044332,-0.080123,0.957555,0.0


In [129]:
from torch.utils.data import TensorDataset
import torch

X_train_ndarray = X_train.values
y_train_ndarray = y_train.values

train_dataset = TensorDataset(torch.from_numpy(X_train_ndarray).type(torch.FloatTensor), torch.from_numpy(y_train_ndarray).type(torch.LongTensor))

# for X_train_temp, y_train_temp in train_dataset:
#     print(X_train_temp, y_train_temp)
#     print(X_train_temp.dtype, y_train_temp.dtype)

In [130]:
X_cv_ndarray = X_cv.values
y_cv_ndarray = y_cv.values

cv_dataset = TensorDataset(torch.from_numpy(X_cv_ndarray).type(torch.FloatTensor), torch.from_numpy(y_cv.values).type(torch.LongTensor))

# for X_cv_temp, y_cv_temp in cv_dataset:
#     print(X_cv_temp, y_cv_temp)
#     print(X_cv_temp.dtype, y_cv_temp.dtype)

In [131]:
from torch.utils.data import DataLoader


train_dataloader = DataLoader(
    dataset=train_dataset,
    batch_size=para.batch_size,
    shuffle=True
)

cv_dataloader = DataLoader(
    dataset=cv_dataset,
    batch_size=para.batch_size,
    shuffle=True
)

### 构建测试集

In [132]:
# data_test = None
# for i_month in para.month_test:
#
#     file_name = para.data_path + '/' + str(i_month) + '.csv'
#     data_curr_month = pd.read_csv(file_name)
#
#     data_curr_month = data_curr_month.dropna(axis=0)
#
#     data_curr_month = label_data(data=data_curr_month, percent_select=para.percent_select)
#
#     if i_month == para.month_test[0]:
#         data_test = data_curr_month
#     else:
#         data_test = pd.concat([data_test, data_curr_month])
#         # data_test = data_test.append(data_curr_month)
#
# X_test = data_test.loc[:, para.feature_column_start_name: para.feature_column_end_name]
# y_test = data_test.loc[:, 'return_bin']

In [133]:
# from torch.utils.data import TensorDataset
# import torch
#
#
# X_test_ndarray = X_test.values
# y_test_ndarray = y_test.values
#
# test_dataset = TensorDataset(torch.from_numpy(X_test_ndarray).type(torch.FloatTensor), torch.from_numpy(y_test_ndarray).type(torch.LongTensor))

In [134]:
# from torch.utils.data import DataLoader
#
#
# test_dataloader = DataLoader(
#     dataset=test_dataset,
#     batch_size=para.batch_size,
#     shuffle=True,
# )

### 构建神经网络

In [135]:
from my_utils.model_class import MLP

model = MLP(in_nums=len(X_train.columns), out_nums=para.classification, drop_p=para.drop)
# to device
model = model.to(device=para.device)
print(model)

MLP(
  (linear_stack): Sequential(
    (0): Linear(in_features=16, out_features=16, bias=True)
    (1): Dropout(p=0.5, inplace=False)
    (2): ReLU()
    (3): Linear(in_features=16, out_features=8, bias=True)
    (4): Dropout(p=0.5, inplace=False)
    (5): ReLU()
    (6): Linear(in_features=8, out_features=2, bias=True)
  )
)


### 训练与测试

In [136]:
import torchmetrics


def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    train_loss, correct = 0, 0

    # initialize metric
    train_precision = torchmetrics.Precision(average='none', num_classes=para.classification).to(device=para.device)

    for batch, (X, y) in enumerate(dataloader):

        # to device
        X = X.to(device=para.device)
        y = y.to(device=para.device)

        # compute prediction and loss
        pred = model(X)
        loss = loss_fn(pred, y)

        train_loss += loss_fn(pred, y).item()
        correct += (pred.argmax(1) == y).type(torch.float).sum().item()

        # metric on current batch
        train_precision(pred.argmax(1), y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # if batch % 10 == 0:
        #     loss, current = loss.item(), batch * len(X)
        #     print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

    train_loss /= num_batches
    correct /= size
    print(f"Train Error: \n    Accuracy: {(100*correct):>0.1f}%, Avg loss: {train_loss:>8f} \n")

    # metric on all batches using custom accumulation
    total_precision = train_precision.compute()
    print("Precision of every train dataset class: ", total_precision)
    print()

    return correct, train_loss, total_precision


def test_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    # initialize metric
    test_precision = torchmetrics.Precision(average='none', num_classes=para.classification).to(device=para.device)

    with torch.no_grad():
        for X, y in dataloader:

            # to device
            X = X.to(device=para.device)
            y = y.to(device=para.device)

            # compute prediction and loss
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

            # metric on current batch
            test_precision(pred.argmax(1), y)


    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n    Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

    # metric on all batches using custom accumulation
    total_precision = test_precision.compute()
    print("Precision of every test dataset class: ", total_precision)
    print()

    return correct, test_loss, total_precision

In [137]:
def select_df_to_dataloader(df: pd.DataFrame, select: int) -> DataLoader:

    df = df[df['return_bin'] == select]

    df_dataset = TensorDataset(
        torch.from_numpy(df.loc[:, para.feature_column_start_name: para.feature_column_end_name].values).type(torch.FloatTensor),
        torch.from_numpy(df.loc[:, 'return_bin'].values).type(torch.LongTensor))

    df_dataloader = DataLoader(
        dataset=df_dataset,
        batch_size=para.batch_size,
        shuffle=True,
    )

    return df_dataloader

In [138]:
# temp2_dataloader = select_df_to_dataloader(df=data_cv, select=2)
temp1_dataloader = select_df_to_dataloader(df=data_cv, select=1)
temp0_dataloader = select_df_to_dataloader(df=data_cv, select=0)

In [139]:
from torch import nn
import time
from torch.utils.tensorboard import SummaryWriter

# 计时
time_start = time.time()

# writer = SummaryWriter(para.tensor_board_log_dir)
writer = SummaryWriter()

# 损失函数
loss_fn = nn.CrossEntropyLoss()
# to device
loss_fn = loss_fn.to(device=para.device)

# 优化器
optimizer = torch.optim.Adam(model.parameters(), lr=para.lr)

epochs = para.epochs
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")

    model.train()
    accuracy_train, loss_train, precision_train = train_loop(train_dataloader, model, loss_fn, optimizer)
    model.eval()
    accuracy_cv, loss_cv, precision_cv = test_loop(cv_dataloader, model, loss_fn)

    # accuracy2 = test_loop(temp2_dataloader, model, loss_fn)
    # print('#')
    # accuracy1 = test_loop(temp1_dataloader, model, loss_fn)
    # accuracy0 = test_loop(temp0_dataloader, model, loss_fn)

    # 写入 tensorboard
    if para.classification == 2:

        writer.add_scalars(main_tag=para.info_str+'_evaluation/cv',
                           tag_scalar_dict={
                               'accuracy': accuracy_cv,
                               'precision0': precision_cv[0],
                               'precision1': precision_cv[1]},
                           global_step=t)

        writer.add_scalars(main_tag=para.info_str+'_evaluation/train',
                           tag_scalar_dict={
                               'accuracy': accuracy_train,
                               'precision0': precision_train[0],
                               'precision1': precision_train[1]},
                           global_step=t)

    elif para.classification == 3:

        writer.add_scalars(main_tag=para.info_str+'_evaluation/cv',
                           tag_scalar_dict={
                               'accuracy': accuracy_cv,
                               'precision0': precision_cv[0],
                               'precision1': precision_cv[1],
                               'precision2': precision_cv[2]},
                           global_step=t)

        writer.add_scalars(main_tag=para.info_str+'_evaluation/train',
                           tag_scalar_dict={
                               'accuracy': accuracy_train,
                               'precision0': precision_train[0],
                               'precision1': precision_train[1],
                               'precision2': precision_train[2]},
                           global_step=t)

    elif para.classification == 5:

        writer.add_scalars(main_tag=para.info_str+'_evaluation/cv',
                           tag_scalar_dict={
                               'accuracy': accuracy_cv,
                               'precision0': precision_cv[0],
                               'precision1': precision_cv[1],
                               'precision2': precision_cv[2],
                               'precision3': precision_cv[3],
                               'precision4': precision_cv[4]},
                           global_step=t)

        writer.add_scalars(main_tag=para.info_str+'_evaluation/train',
                           tag_scalar_dict={
                               'accuracy': accuracy_train,
                               'precision0': precision_train[0],
                               'precision1': precision_train[1],
                               'precision2': precision_train[2],
                               'precision3': precision_train[3],
                               'precision4': precision_train[4]},
                           global_step=t)

    writer.add_scalars(main_tag=para.info_str+'_loss/cv',
                       tag_scalar_dict={
                           'loss': loss_cv},
                       global_step=t)

    writer.add_scalars(main_tag=para.info_str+'_loss/train',
                       tag_scalar_dict={
                           'loss': loss_train},
                       global_step=t)
    writer.flush()

    time_end = time.time()
    print('Time cost = %fs' % (time_end - time_start))
    print()

writer.close()

print("Done!")

Epoch 1
-------------------------------
Train Error: 
    Accuracy: 50.7%, Avg loss: 0.695929 

Precision of every train dataset class:  tensor([0.4994, 0.5128], device='cuda:0')

Test Error: 
    Accuracy: 58.9%, Avg loss: 0.688260 

Precision of every test dataset class:  tensor([0.6667, 0.5769], device='cuda:0')

Time cost = 1.773974s

Epoch 2
-------------------------------
Train Error: 
    Accuracy: 51.3%, Avg loss: 0.693084 

Precision of every train dataset class:  tensor([0.5093, 0.5151], device='cuda:0')

Test Error: 
    Accuracy: 53.1%, Avg loss: 0.684278 

Precision of every test dataset class:  tensor([0.4286, 0.5409], device='cuda:0')

Time cost = 3.391373s

Epoch 3
-------------------------------
Train Error: 
    Accuracy: 53.5%, Avg loss: 0.685561 

Precision of every train dataset class:  tensor([0.5448, 0.5303], device='cuda:0')

Test Error: 
    Accuracy: 59.3%, Avg loss: 0.685718 

Precision of every test dataset class:  tensor([0.5769, 0.6012], device='cuda:0')



## 保存模型

In [140]:
torch.save(model.state_dict(), para.save_model_path)

print('Finish save model!')

Finish save model!


## captum

In [141]:
# # captum
# from captum.attr import IntegratedGradients
#
# ig = IntegratedGradients(model)

In [142]:
# temp = cv_dataloader.dataset.tensors[0]
# temp.requires_grad_()
# attr, delta = ig.attribute(temp,target=1, return_convergence_delta=True)
# attr = attr.detach().numpy()

In [143]:
# # Helper method to print importances and visualize distribution
# def visualize_importances(feature_names, importances, title="Average Feature Importances", plot=True, axis_title="Features"):
#     print(title)
#     for i in range(len(feature_names)):
#         print(feature_names[i], ": ", '%.3f'%(importances[i]))
#     y_pos = (np.arange(len(feature_names)))
#     if plot:
#         plt.figure(figsize=(20,6))
#         plt.barh(y_pos, importances, align='center')
#         plt.yticks(y_pos, feature_names)
#         plt.ylabel(axis_title)
#         plt.grid(axis='y')
#         plt.title(title)
# visualize_importances(feature_names=X_cv.columns.values.tolist(), importances=np.mean(attr, axis=0))

In [144]:
# X_cv.columns.values.tolist()

In [145]:
# loss = nn.CrossEntropyLoss()
# input = torch.Tensor(
#     [[-0.0441,  0.0773],
#     [-0.0781, -0.1772],
#     [-0.1319, -0.0432],
#     [-0.0714, -0.1261],
#     [-0.0806, -0.1370],
#     [-0.1730, -0.1472],
#     [-0.0350, -0.0507],
#     [-0.1149, -0.2248]])
# # input = input.reshape(-1,4)
# target = torch.Tensor([0, 1, 1, 0, 0, 0, 0, 0]).type(torch.LongTensor)
# print(input.dtype)
# print(target.dtype)
# output = loss(input, target)
# print(input, target, output)

In [146]:
# # Example of target with class indices
# loss = nn.CrossEntropyLoss()
# input = torch.randn(3, 5, requires_grad=True)
# target = torch.Tensor([1,4,1]).type(torch.LongTensor)
# output = loss(input, target)
# print(input,target,output)

In [147]:
# loss = nn.BCEWithLogitsLoss()
# input = torch.Tensor([0.5, 0.4, 0.3])
# target = torch.Tensor([0])
# output = loss(input, target)
# print(input, target, output)